In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

# Load dataset
df = pd.read_csv(r"C:\Users\Tian\Desktop\NLP_Task7\Datasets\Corona_NLP_train.csv", encoding='latin1')
df = df[['OriginalTweet', 'Sentiment']].dropna()
df = df[df['Sentiment'].isin(['Positive', 'Negative', 'Neutral'])] 

: 

In [ ]:
# Encode labels
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df['label'] = df['Sentiment'].map(label_map)

# Preprocess
df['clean_text'] = df['OriginalTweet'].str.lower().str.replace(r"[^\w\s]", "", regex=True)
tokenized_texts = df['clean_text'].str.split()
word_counts = Counter(word for sentence in tokenized_texts for word in sentence)


In [ ]:
# Build vocab
vocab = {word: i + 2 for i, word in enumerate(word_counts)}
vocab['<PAD>'] = 0
vocab['<UNK>'] = 1

def text_to_seq(text):
    return [vocab.get(word, vocab['<UNK>']) for word in text]

# Convert to tensors
X_seq = [torch.tensor(text_to_seq(sentence)) for sentence in tokenized_texts]
y_seq = torch.tensor(df['label'].values)
X_padded = pad_sequence(X_seq, batch_first=True, padding_value=vocab['<PAD>'])

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_seq, test_size=0.2, random_state=42)

# Dataset wrapper
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_loader = DataLoader(TextDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TextDataset(X_test, y_test), batch_size=64)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, kernel_sizes=[3, 4, 5], num_filters=100):
        super(TextCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab['<PAD>'])
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (k, embed_dim)) for k in kernel_sizes
        ])
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

    def forward(self, x):
        x = self.embedding(x)  # (batch, seq_len, embed_dim)
        x = x.unsqueeze(1)  # (batch, 1, seq_len, embed_dim)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(batch, num_filters, seq_len - k + 1), ...]
        x = [F.max_pool1d(c, c.size(2)).squeeze(2) for c in x]  # [(batch, num_filters), ...]
        x = torch.cat(x, dim=1)  # (batch, num_filters * len(kernel_sizes))
        x = self.dropout(x)
        return self.fc(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = TextCNN(vocab_size=len(vocab), embed_dim=100, num_classes=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

epochs = 5
model.train()

for epoch in range(epochs):
    total_loss, correct, total = 0, 0, 0
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = outputs.argmax(dim=1)
        correct += (predictions == batch_y).sum().item()
        total += batch_y.size(0)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Accuracy: {correct/total:.4f}")

In [ ]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        outputs = model(batch_x)
        predictions = outputs.argmax(dim=1)
        correct += (predictions == batch_y).sum().item()
        total += batch_y.size(0)

print(f"Test Accuracy: {correct / total:.4f}")